# Implied Volatilities

In this notebook we illustrate calculated implied volatilities from option data.

In [ ]:
import sys
import warnings

sys.path.append('../python/')
warnings.simplefilter(action='ignore', category=UserWarning)  # suppress pandas sql warning

import numpy as np
import pandas as pd
import plotly.graph_objects as go

import data
data.initialise()

from smileplot import smile_plot
from smiledata import smile_data


In [ ]:
date = '2022-09-16'
symbols = pd.read_csv('../data/DJIA.csv', sep=';')['Symbol']

In [ ]:
# df = smile_data('SHW', date)
# df

In [ ]:
for symbol in symbols:
    try:
        smile_plot(symbol, date).show()
    except:
        print("Error plotting " + symbol + ".")
